In [1]:
import requests

In [2]:
import pandas as pd

In [4]:
calls_list = pd.read_csv("https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2025-07/DVRCA_2001-2024.csv")

In [6]:
calls_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199370 entries, 0 to 199369
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   YEAR                199370 non-null  int64  
 1   COUNTY              199370 non-null  object 
 2   AGENCY_NAME         199370 non-null  object 
 3   MONTH               199370 non-null  int64  
 4   TOTAL_CALLS         199370 non-null  int64  
 5   WEAPONS_INVOLVED    199370 non-null  int64  
 6   SUB_FIREARM         199370 non-null  int64  
 7   SUB_KNIFE           199370 non-null  int64  
 8   SUB_OTHER           199370 non-null  int64  
 9   SUB_PERSONAL        199370 non-null  int64  
 10  WEAPON_NOT_RPT      199370 non-null  int64  
 11  TOTAL_STRANG_SUFFO  58974 non-null   float64
 12  SUB_STRANGULATION   58974 non-null   float64
 13  SUB_SUFFOCATION     58974 non-null   float64
dtypes: float64(3), int64(9), object(2)
memory usage: 21.3+ MB


In [12]:
# Keep only rows where YEAR is 2015 or later
calls_list_pastdecade = calls_list[calls_list['YEAR'] >= 2015]
#calls_list_pastdecade.info()

In [18]:
calls_list_pastdecade.groupby("YEAR")["TOTAL_CALLS"].sum()


YEAR
2015    162302
2016    164569
2017    169362
2018    166890
2019    161123
2020    160646
2021    164608
2022    162422
2023    160357
2024    163024
Name: TOTAL_CALLS, dtype: int64

In [ ]:
# statewide trends for DV calls
statewide_trends = calls_list_pastdecade.groupby('YEAR').agg({
    'TOTAL_CALLS': 'sum',
    'WEAPONS_INVOLVED': 'sum'
}).reset_index()
statewide_trends.to_csv('statewide_trends.csv', index=False)

In [25]:
# weapon types calculation across all years
weapon_summary = calls_list_pastdecade.agg({
    'TOTAL_CALLS': 'sum',
    'WEAPONS_INVOLVED': 'sum',
    'SUB_FIREARM': 'sum',
    'SUB_KNIFE': 'sum',
    'SUB_PERSONAL': 'sum',
    'SUB_OTHER': 'sum',
    'WEAPON_NOT_RPT': 'sum'
})

# Create a dataframe/table for weapon types
weapon_data = pd.DataFrame({
    'Weapon_Type': ['Firearm', 'Knife', 'Personal Weapons', 'Other', 'Not Reported'],
    'Count': [
        weapon_summary['SUB_FIREARM'],
        weapon_summary['SUB_KNIFE'],
        weapon_summary['SUB_PERSONAL'],
        weapon_summary['SUB_OTHER'],
        weapon_summary['WEAPON_NOT_RPT']
    ]
})

weapon_data['Total_Calls'] = weapon_summary['TOTAL_CALLS']
weapon_data['Calls_With_Weapons'] = weapon_summary['WEAPONS_INVOLVED']
weapon_data['Calls_Without_Weapons'] = weapon_data['Total_Calls'] - weapon_data['Calls_With_Weapons']

weapon_data.to_csv('weapon_involvement.csv', index=False)

In [87]:
# total and average DV calls by county
county_stats = calls_list_pastdecade.groupby("COUNTY")["TOTAL_CALLS"].sum().reset_index()
county_stats["AVG_ANNUAL_CALLS"] = (county_stats["TOTAL_CALLS"] / 10).round(0).astype(int)
#print(county_stats)

In [88]:
pop_df = pd.read_csv("county_population.csv")

In [89]:
county_stats = pd.merge(county_stats, pop_df, on="COUNTY")
county_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   COUNTY            58 non-null     object
 1   TOTAL_CALLS       58 non-null     int64 
 2   AVG_ANNUAL_CALLS  58 non-null     int64 
 3   POPULATION        58 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ KB


In [90]:
county_stats["CALL_RATE_PER_100K"] = (county_stats["AVG_ANNUAL_CALLS"] / county_stats["POPULATION"] * 100000).round(0).astype(int)

In [91]:
county_stats.to_csv("county_dv_call_rate.csv", index=False)
